In [1]:
import json
from bs4 import Tag
from tqdm import tqdm
import tb1025_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

# def read_json(path, file_name):
#     try:
#         with open(path+file_name, 'r', encoding='utf-8') as openfile:
#             Loaded_file = json.load(openfile)
#             print(f"Successfully loaded: {file_name}")
#         return Loaded_file
#     except Exception as e:
#         print(f"Error loading {file_name}: {str(e)}")
#         return None

def scrape_article(url, page_key_code):
    try:
        article_content = tb1025_utils.scrape_tb1025_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    all_article = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        start = 1
        for i in range(start, Total_lenght+1):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_kbcmw_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"tb1025_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            
            page_key_code = "Page "+page_key_list[1]+" "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [3]:
%%time
# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in tb1025_ALL_link_གསར་འགྱུར།.json: 2
page key name: Page གསར་འགྱུར། 2
Page key code: Page གསར་འགྱུར། 
Total page in tb1025_ALL_link_ཆོས་འབྱུང་།.json: 16
page key name: Page ཆོས་འབྱུང་། 16
Page key code: Page ཆོས་འབྱུང་། 
Total page in tb1025_ALL_link_བཀའ་འགྱུར།.json: 7
page key name: Page བཀའ་འགྱུར། 7
Page key code: Page བཀའ་འགྱུར། 
Total page in tb1025_ALL_link_བསྟན་འགྱུར།.json: 6
page key name: Page བསྟན་འགྱུར། 6
Page key code: Page བསྟན་འགྱུར། 
Total page in tb1025_ALL_link_ཚད་མ།.json: 3
page key name: Page ཚད་མ། 3
Page key code: Page ཚད་མ། 
Total page in tb1025_ALL_link_ཕར་ཕྱིན།.json: 11
page key name: Page ཕར་ཕྱིན། 11
Page key code: Page ཕར་ཕྱིན། 
Total page in tb1025_ALL_link_དབུ་མ།.json: 2
page key name: Page དབུ་མ། 2
Page key code: Page དབུ་མ། 
Total page in tb1025_ALL_link_ཆོ་ག.json: 2
page key name: Page ཆོ་ག 2
Page key code: Page ཆོ་ག 
Total page in tb1025_ALL_link_སྔགས།.json: 2
page key name: Page སྔགས། 2
Page key code: Page སྔགས། 
Total page in tb1025_ALL_link_

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]


  0%|          | 0/26 [00:00<?, ?it/s]



  0%|          | 0/32 [00:00<?, ?it/s]




  0%|          | 0/128 [00:00<?, ?it/s]





  0%|          | 0/60 [00:00<?, ?it/s]






  0%|          | 0/183 [00:00<?, ?it/s]







  0%|          | 0/151 [00:00<?, ?it/s]









  0%|          | 0/400 [00:00<?, ?it/s]








  0%|          | 0/255 [00:00<?, ?it/s]


  4%|▍         | 1/26 [00:06<02:34,  6.19s/it]

  2%|▏         | 1/49 [00:27<18:33, 23.20s/it]







  1%|          | 1/151 [00:29<1:01:09, 24.46s/it]







  1%|▏         | 2/151 [00:32<36:15, 14.60s/it]  







  2%|▏         | 3/151 [00:39<26:32, 10.76s/it]







  3%|▎         | 4/151 [00:47<24:11,  9.87s/it]







  3%|▎         | 5/151 [00:51<17:48,  7.32s/it]







  4%|▍         | 6/151 [00:53<14:28,  5.99s/it]








  0%|          | 1/255 [00:55<2:03:48, 29.25s/it]



  5%|▌         | 2/37 [00:56<16:40, 28.60s/it]


  8%|▊         | 2/

Total Failure in the དབུ་མ། article: 26
tb1025_ALL_content_དབུ་མ།.json
Successfully saved: tb1025_ALL_content_དབུ་མ།.json
Total page in tb1025_ALL_link_རྒྱུན་ཤེས་དམར་ཁྲིད།.json: 4
page key name: Page རྒྱུན་ཤེས་དམར་ཁྲིད། 4
Page key code: Page རྒྱུན་ཤེས་དམར་ཁྲིད། 


 92%|█████████▏| 34/37 [01:50<00:05,  1.92s/it]





 47%|████▋     | 28/60 [01:51<00:46,  1.47s/it]




 26%|██▌       | 33/128 [01:55<01:35,  1.01s/it]






  5%|▍         | 9/183 [01:56<17:17,  5.96s/it]






  5%|▌         | 10/183 [01:56<25:18,  8.78s/it]

 61%|██████    | 30/49 [01:56<00:40,  2.14s/it]









  9%|▉         | 37/400 [01:56<06:30,  1.08s/it]









100%|██████████| 37/37 [01:56<00:00,  2.05s/it]




 27%|██▋       | 34/128 [01:56<02:13,  1.42s/it]




 27%|██▋       | 35/128 [01:56<02:06,  1.36s/it]

 63%|██████▎   | 31/49 [01:57<00:53,  2.98s/it]

 65%|██████▌   | 32/49 [01:57<00:45,  2.68s/it]





 48%|████▊     | 29/60 [01:57<01:23,  2.69s/it]





 50%|█████     | 30/60 [01:57<01:23,  2.79s/it]





 52%|█████▏    | 31/60 [01:57<01:10,  2.43s/it]








  8%|▊         | 21/255 [01:57<12:35,  3.23s/it]




 30%|██▉       | 38/128 [01:58<01:25,  1.05it/s]







 28%|██▊       | 43/151 [01:58<02:00,  1.11s/it]







 29%|██▉       | 44/151 [01:58<02:14,

Total Failure in the ཆོ་ག article: 32
tb1025_ALL_content_ཆོ་ག.json


...

Total Failure in the བོད་ལྗོངས། article: 16
tb1025_ALL_content_བོད་ལྗོངས།.json
Successfully saved: tb1025_ALL_content_བོད་ལྗོངས།.json












 96%|█████████▋| 386/400 [16:08<00:04,  2.87it/s]









 97%|█████████▋| 387/400 [16:09<00:04,  3.05it/s]





 59%|█████▉    | 195/328 [06:38<00:55,  2.40it/s]





 60%|█████▉    | 196/328 [06:38<00:45,  2.92it/s]









 97%|█████████▋| 388/400 [16:09<00:04,  2.81it/s]





 60%|██████    | 197/328 [06:38<00:42,  3.12it/s]









 97%|█████████▋| 389/400 [16:10<00:04,  2.52it/s]





 60%|██████    | 198/328 [06:39<00:44,  2.90it/s]





 61%|██████    | 199/328 [06:39<00:36,  3.54it/s]





 61%|██████    | 200/328 [06:39<00:33,  3.84it/s]





 62%|██████▏   | 202/328 [06:39<00:20,  6.02it/s]









 98%|█████████▊| 392/400 [16:10<00:02,  3.13it/s]





 62%|██████▏   | 203/328 [06:39<00:25,  4.87it/s]





 62%|██████▏   | 204/328 [06:39<00:22,  5.40it/s]









 98%|█████████▊| 394/400 [16:11<00:01,  3.58it/s]









 99%|█████████▉| 395/400 [16:11<00:01,  4.04it/s]





 62%|██████▎   | 205/328 [06:40<00:42,  2.86it/s]









 99%|█████████▉| 396/400 [1

Total Failure in the ཆོས་འབྱུང་། article: 384
tb1025_ALL_content_ཆོས་འབྱུང་།.json
Successfully saved: tb1025_ALL_content_ཆོས་འབྱུང་།.json








 65%|██████▌   | 214/328 [06:43<00:42,  2.69it/s]





 66%|██████▌   | 215/328 [06:43<00:38,  2.90it/s]





 66%|██████▌   | 216/328 [06:44<00:37,  3.01it/s]





 66%|██████▌   | 217/328 [06:44<00:31,  3.48it/s]





 66%|██████▋   | 218/328 [06:44<00:35,  3.12it/s]





 67%|██████▋   | 219/328 [06:44<00:34,  3.12it/s]





 67%|██████▋   | 220/328 [06:45<00:33,  3.25it/s]





 68%|██████▊   | 222/328 [06:45<00:28,  3.78it/s]





 68%|██████▊   | 223/328 [06:45<00:25,  4.10it/s]





 69%|██████▊   | 225/328 [06:46<00:23,  4.47it/s]





 69%|██████▉   | 227/328 [06:46<00:27,  3.67it/s]





 70%|██████▉   | 228/328 [06:47<00:27,  3.66it/s]





 70%|██████▉   | 229/328 [06:47<00:36,  2.68it/s]





 70%|███████   | 231/328 [06:48<00:30,  3.16it/s]





 71%|███████▏  | 234/328 [06:48<00:21,  4.45it/s]





 72%|███████▏  | 235/328 [06:49<00:34,  2.72it/s]





 72%|███████▏  | 237/328 [06:50<00:30,  2.98it/s]





 73%|███████▎  | 239/328 [06:50<00:24,  3.65it/s]





 73%

Total Failure in the གསེར་ཁྲི། article: 321
tb1025_ALL_content_གསེར་ཁྲི།.json
Successfully saved: tb1025_ALL_content_གསེར་ཁྲི།.json
Processed 63 files
CPU times: user 1h 16min 10s, sys: 29.8 s, total: 1h 16min 40s
Wall time: 16min 48s


In [ ]:
# 14:44

In [ ]:
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files